In [40]:
from datetime import date
import calendar as c
import pandas as pd
import numpy as np

In [79]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1500)
pd.options.display.max_colwidth = 100000

#### Para testear las funciones, levanto los dataframes originales

In [41]:
df_pageviews = pd.read_csv('data/pageviews.csv', parse_dates=['FEC_EVENT'])
df_devicedata = pd.read_csv('data/device_data.csv', parse_dates=["FEC_EVENT"])
year = 2018
mes_snapshot = 9

def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

universo = universo_train(df_pageviews)

In [42]:
def _sum_campo_user(df_pageviews, campo):
    temp = pd.crosstab(df_pageviews.USER_ID, df_pageviews[campo]) 
    temp.columns = [campo + "_" + str(v) for v in temp.columns]
    return _df_refactor(temp)

In [76]:
def _df_refactor(df_temp):
    df_union = pd.merge(universo, df_temp, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = df_union.drop(['snapshot_mes'], axis=1)
    df_union = df_union.sort_values('user_id')
    return df_union

In [63]:
def _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_campo_user(df, campo)

In [86]:
def _sum_total_user(df_pageviews, campo):
    temp = df_pageviews.groupby("USER_ID")[campo].sum().to_frame().reset_index()
    temp = _df_refactor(temp)
    temp = temp.drop(['USER_ID'], axis=1)
    return temp

In [46]:
def _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_total_user(df, campo)

In [47]:
def _sum_user_campo_all(df_pageviews, mes_snapshot, campo):
    fecha_desde_3m = pd.Timestamp(year, mes_snapshot - 2, 1)
    fecha_desde_1m = pd.Timestamp(year, mes_snapshot, 1)
    fecha_desde_15d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 15)
    fecha_desde_7d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 7)
    fecha_desde_1d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 1)
    fecha_hasta_last_day = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    
    df_3m = _sum_campo_user_between(df_pageviews, fecha_desde_3m, fecha_hasta_last_day, campo)
    df_1m = _sum_campo_user_between(df_pageviews, fecha_desde_1m, fecha_hasta_last_day, campo)
    df_15d = _sum_campo_user_between(df_pageviews, fecha_desde_15d, fecha_hasta_last_day, campo)
    df_7d = _sum_campo_user_between(df_pageviews, fecha_desde_7d, fecha_hasta_last_day, campo)
    df_1d =_sum_campo_user_between(df_pageviews, fecha_desde_1d, fecha_hasta_last_day, campo)
    df_ever =_sum_campo_user(df_pageviews, campo)

    tot_3m = _sum_total_user_between(df_pageviews, fecha_desde_3m, fecha_hasta_last_day, campo)
    tot_1m = _sum_total_user_between(df_pageviews, fecha_desde_1m, fecha_hasta_last_day, campo)
    tot_15d = _sum_total_user_between(df_pageviews, fecha_desde_15d, fecha_hasta_last_day, campo)
    tot_7d = _sum_total_user_between(df_pageviews, fecha_desde_7d, fecha_hasta_last_day, campo)    
    tot_1d = _sum_total_user_between(df_pageviews, fecha_desde_1d, fecha_hasta_last_day, campo)
    tot_ever = _sum_total_user(df_pageviews, campo)

    return df_3m, df_1m, df_15d, df_7d, df_1d, df_ever, tot_3m, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever

In [132]:
def _ratio_campo_df(df_numerador, df_denominador, name):
    #Equiparar columnas
    ######COMPLETAR####
    
    #Eliminar columnar user_id para que no la divida
    users = df_numerador['user_id']
    df_numerador = df_numerador.drop(['user_id'], axis = 1)
    df_denominador = df_denominador.drop(['user_id'], axis = 1)

    #División element-wise
    temp = df_numerador.div(df_denominador, axis=0, level = 'user_id', fill_value = 0)
    temp.columns = [name + "_" + str(v) for v in temp.columns]

    #Agrega columna user_id
    temp['user_id'] = users

    #Completa Nan
    temp = temp.fillna(-999)
    return temp

In [106]:
def _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo):
    df_3m, df_1m, df_15d, df_7d, df_1d, df_ever, tot_3m, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever = _sum_user_campo_all(df_pageviews, mes_snapshot, campo)
    
    ratio_preferencia_3m = _ratio_campo_df(df_3m, tot_3m, 'ratio_preferencia_3m')
    ratio_preferencia_1m = _ratio_campo_df(df_1m, tot_1m, 'ratio_preferencia_1m')
    ratio_preferencia_15d = _ratio_campo_df(df_15d, tot_15d, 'ratio_preferencia_15d')
    ratio_preferencia_7d = _ratio_campo_df(df_7d, tot_7d, 'ratio_preferencia_7d')
    ratio_preferencia_1d = _ratio_campo_df(df_1d, tot_1d, 'ratio_preferencia_1d')
    
    ratio_interes_3m = _ratio_campo_df(df_3m, df_ever, 'ratio_interes_3m')
    ratio_interes_1m = _ratio_campo_df(df_1m, df_ever, 'ratio_interes_1m')
    ratio_interes_15d = _ratio_campo_df(df_15d, df_ever, 'ratio_interes_15d')
    ratio_interes_7d = _ratio_campo_df(df_7d, df_ever, 'ratio_interes_7d')
    ratio_interes_1d = _ratio_campo_df(df_1d, df_ever, 'ratio_interes_1d')
    
    ratio_interes_reciente_1m = _ratio_campo_df(df_1m, df_3m, 'ratio_interes_reciente_1m')
    ratio_interes_reciente_15d = _ratio_campo_df(df_15d, df_3m, 'ratio_interes_reciente_15d')
    ratio_interes_reciente_7d = _ratio_campo_df(df_7d, df_3m, 'ratio_interes_reciente_7d')
    ratio_interes_reciente_1d = _ratio_campo_df(df_1d, df_3m, 'ratio_interes_reciente_1d')

    ratio_actividad_3m = _ratio_campo_df(tot_3m, tot_ever, 'ratio_actividad_3m')
    ratio_actividad_1m = _ratio_campo_df(tot_1m, tot_ever, 'ratio_actividad_1m')
    ratio_actividad_15d = _ratio_campo_df(tot_15d, tot_3m, 'ratio_actividad_15d')
    ratio_actividad_7d = _ratio_campo_df(tot_7d, tot_3m, 'ratio_actividad_7d')
    ratio_actividad_1d = _ratio_campo_df(tot_1d, tot_3m, 'ratio_actividad_1d')
    
    df_union = pd.merge(universo, ratio_preferencia_3m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    
    df_union = pd.merge(df_union, ratio_interes_3m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    
    df_union = pd.merge(df_union, ratio_interes_reciente_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_1d, how='left', left_on=['user_id'], right_on=['user_id'])

    df_union = pd.merge(df_union, ratio_actividad_3m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    
    return df_union

In [119]:
#PROBAR
temp = ratio_page_all(universo, df_pageviews, mes_snapshot)

KeyError: 'user_id'

In [ ]:
temp.head(10)

In [120]:
df_3m, df_1m, df_15d, df_7d, df_1d, df_ever, tot_3m, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever = _sum_user_campo_all(df_pageviews, mes_snapshot, 'PAGE')

In [115]:
tot_3m.head(10)

,user_id,PAGE
0,0,57360.0
1,1,11209.0
2,2,41080.0
3,3,18370.0
4,4,33837.0
5,5,72809.0
6,6,22381.0
7,7,28169.0
8,8,28004.0
9,9,39772.0


In [126]:
temp = df_3m.div(df_ever, axis=0, level = 'user_id', fill_value = 0)
temp.columns = ['ratio_preferencia_3m' + "_" + str(v) for v in temp.columns]

In [127]:
temp.head(10)

ratio_preferencia_3m_PAGE_1  ratio_preferencia_3m_PAGE_10  \
0                     0.131868                      0.237449   
1                     0.155251                      0.151292   
2                     0.350649                      0.140845   
3                     0.200000                           NaN   
4                     0.151515                      0.142336   
5                     0.000000                           NaN   
6                     0.333333                      0.571429   
7                     0.159292                      0.186761   
8                     0.000000                      0.070922   
9                     0.416667                      0.163498   

   ratio_preferencia_3m_PAGE_100  ratio_preferencia_3m_PAGE_1000  \
0                            NaN                             NaN   
1                            NaN                             NaN   
2                            NaN                             NaN   
3                            NaN                             NaN   
4                            0.0                             1.0   
5                            NaN                             NaN   
6                            NaN                             NaN   
7                            NaN                             NaN   
8                            NaN                             NaN   
9                            NaN                             NaN   

   ratio_preferencia_3m_PAGE_1001  ratio_preferencia_3m_PAGE_1002  \
0                             NaN                             NaN   
1                             NaN                             NaN   
2                             NaN                             NaN   
3                             NaN                             NaN   
4                             NaN                             NaN   
5                             NaN                             NaN   
6                             NaN                             NaN   
7                             NaN                             NaN   
8                             NaN                             NaN   
9                             NaN                             NaN   

   ratio_preferencia_3m_PAGE_1003  ratio_preferencia_3m_PAGE_1004  \
0                             NaN                             NaN   
1                             NaN                             NaN   
2                             NaN                             NaN   
3                             NaN                             NaN   
4                             NaN                             NaN   
5                             NaN                             NaN   
6                             NaN                             NaN   
7                             NaN                             NaN   
8                             NaN                             NaN   
9                             NaN                             NaN   

   ratio_preferencia_3m_PAGE_1005  ratio_preferencia_3m_PAGE_1006  \
0                             NaN                             NaN   
1                             NaN                             NaN   
2                             NaN                             NaN   
3                             NaN                             NaN   
4                             NaN                             NaN   
5                             NaN                             NaN   
6                             NaN                             NaN   
7                             NaN                             NaN   
8                             NaN                             NaN   
9                             NaN                             NaN   

   ratio_preferencia_3m_PAGE_1007  ratio_preferencia_3m_PAGE_1008  \
0                             NaN                             NaN   
1                             NaN                             NaN   
2                             NaN                             

In [114]:
df_3m.head(10)

user_id  PAGE_1  PAGE_2  PAGE_3  PAGE_4  PAGE_5  PAGE_6  PAGE_7  PAGE_8  \
0        0    24.0   245.0   132.0    76.0    39.0     9.0     0.0     0.0   
1        1    34.0    63.0    34.0     9.0     2.0     2.0     0.0     0.0   
2        2    27.0   140.0   116.0    25.0    24.0     6.0     0.0     0.0   
3        3     2.0   116.0    93.0    27.0    30.0    10.0     0.0     0.0   
4        4     5.0    77.0    51.0     8.0    40.0     6.0     0.0     0.0   
5        5     0.0   210.0   296.0    75.0    41.0    21.0     1.0     3.0   
6        6     1.0    43.0    33.0     5.0     5.0     3.0     0.0     0.0   
7        7    18.0   134.0    69.0    27.0    20.0     2.0     0.0     0.0   
8        8     0.0   102.0    65.0    35.0     3.0    44.0     4.0     0.0   
9        9     5.0   123.0   135.0    21.0    19.0     1.0     0.0     0.0   

   PAGE_9  PAGE_10  PAGE_11  PAGE_12  PAGE_13  PAGE_14  PAGE_15  PAGE_16  \
0    72.0    175.0    149.0    174.0     32.0     11.0     79.0      0.0   
1    22.0     41.0      0.0     35.0     24.0      0.0     21.0      0.0   
2     7.0     20.0      0.0      0.0      0.0      4.0     11.0      0.0   
3     0.0      0.0      0.0      0.0      0.0      4.0      0.0      0.0   
4     0.0     78.0      0.0      0.0      0.0      0.0      2.0      0.0   
5     0.0      0.0      0.0      0.0      0.0     42.0      0.0      0.0   
6     0.0     72.0      0.0     72.0      0.0      0.0     14.0      0.0   
7    33.0     79.0      0.0      7.0      0.0      4.0     45.0      0.0   
8     5.0     10.0      0.0      0.0      0.0      0.0      3.0      0.0   
9    11.0     43.0      0.0      0.0      0.0     10.0     12.0      0.0   

   PAGE_17  PAGE_18  PAGE_19  PAGE_20  PAGE_21  PAGE_22  PAGE_23  PAGE_24  \
0      3.0     24.0      0.0     94.0     31.0      1.0      6.0     13.0   
1      0.0     35.0      0.0      6.0     13.0      0.0      1.0      0.0   
2      0.0      0.0      0.0     39.0     44.0      0.0      5.0      4.0   
3      0.0      0.0      1.0     21.0     24.0      0.0      3.0      6.0   
4      0.0      0.0      0.0     31.0      0.0      1.0     27.0      2.0   
5      0.0      0.0      2.0     45.0    202.0      0.0      6.0      1.0   
6      0.0     62.0      0.0     14.0     18.0      0.0      2.0     20.0   
7      1.0      7.0      3.0     17.0     36.0      0.0     16.0      2.0   
8      1.0      0.0      0.0     34.0     13.0      0.0     13.0      9.0   
9      1.0      0.0      3.0      8.0     13.0      2.0     50.0      2.0   

   PAGE_25  PAGE_26  PAGE_27  PAGE_28  PAGE_29  PAGE_30  PAGE_31  PAGE_32  \
0     11.0     30.0     72.0      0.0      0.0      0.0      0.0     21.0   
1      0.0      3.0      4.0      0.0      0.0      0.0      0.0      6.0   
2      0.0     20.0     23.0      0.0      0.0      0.0      0.0      3.0   
3      1.0     16.0     24.0      0.0      0.0      1.0      1.0     16.0   
4      0.0      4.0     71.0      0.0      0.0      0.0      3.0      0.0   
5      0.0     23.0     12.0      0.0      0.0      0.0      0.0     67.0   
6      1.0     36.0      6.0      0.0      0.0      0.0      0.0     11.0   
7      1.0     10.0     35.0      0.0      0.0      0.0      0.0      2.0   
8      5.0     20.0     30.0      0.0      0.0      0.0      0.0      4.0   
9      0.0      6.0     14.0      0.0      0.0      0.0      0.0      5.0   

   PAGE_33  PAGE_34  PAGE_35  PAGE_36  PAGE_37  PAGE_38  PAGE_39  PAGE_40  \
0      3.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0      0.0      1.0   
2      0.0      0.0      0.0      0.0      0.0      0.0     27.0     36.0   
3      0.0      0.0      0.0      0.0      0.0      0.0      1.0      6.0   
4      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
5     10.0      0.0      0.0      0.0      0.0      0.0     29.0    111.0   
6      1.0      0.0      0.0      0.0      0.0      0.0      0.0      4.

In [128]:
df_ever.head(10)

user_id  PAGE_1  PAGE_2  PAGE_3  PAGE_4  PAGE_5  PAGE_6  PAGE_7  PAGE_8  \
0        0     182     826     417     279     109      34       1       1   
1        1     219     332     139      29      13       7       0       0   
2        2      77     398     271      61      63       7       0       1   
3        3      10     373     282      67      79      25       0       0   
4        4      33     374     310      27     145      56       0       1   
5        5       3     777     879     229     102      87       1       5   
6        6       3     138     146      18      12      11       0       0   
7        7     113     451     209      89      28       5       0       1   
8        8       4     376     236     106      23      65       6       0   
9        9      12     433     520     124      63       1       0       0   

   PAGE_9  PAGE_10  PAGE_11  PAGE_12  PAGE_13  PAGE_14  PAGE_15  PAGE_16  \
0     286      737      659      729      144      137      328        7   
1     116      271        0      194      126       35      107        0   
2      66      142        0        1        2       25       36        0   
3       0        0        0        0        0      135        0        0   
4       3      548        0        0        0        2       16        0   
5       0        0        0        0        0      257        0        0   
6       3      126        0      126        0       13       27        0   
7     177      423        0       25        0       23      162        0   
8      64      141        0        0        0       21       41        0   
9      25      263        0        0        0      109       82        0   

   PAGE_17  PAGE_18  PAGE_19  PAGE_20  PAGE_21  PAGE_22  PAGE_23  PAGE_24  \
0       12       70        4      358      112       16       16       43   
1        1      194        2       36       82        3        6        1   
2        8        1        0       79      176        4        7        8   
3        0        0        2       41      102        0        9       12   
4        1        0        1       56        3       24      129        7   
5        0        0        7      189      653        0        8       57   
6        0      107        0       32       67        0        5       64   
7        7       25        3       84      134        6       37        3   
8       10        0        0      105       47        2       45       13   
9        2        0        3       42       60        8      168        4   

   PAGE_25  PAGE_26  PAGE_27  PAGE_28  PAGE_29  PAGE_30  PAGE_31  PAGE_32  \
0       36       96      218        1        1        1        1       68   
1        1       26       14        0        0        0        0       32   
2        0       33       56        3        2        0        3       14   
3        2       79       52        0        1        1        2       51   
4        6        8      281        0        0        0        3        0   
5        0      164       23        0        0        0        1      228   
6        2      122       16        0        0        0        0       40   
7        3       42       87        0        0        0        0       14   
8        5       55      135        0        0        0        8       22   
9        2       17       34        0        2        0        4       15   

   PAGE_33  PAGE_34  PAGE_35  PAGE_36  PAGE_37  PAGE_38  PAGE_39  PAGE_40  \
0        7        1        1        4        4        4        8        3   
1        0        1        0        0        0        0        0        4   
2        0        1        0        0        0        0       44      155   
3        0        0        0        0        0        0        3       81   
4        0        0        0        0        0        0        0       13   
5       39        0        0        0        0        0       62      438   
6        2        0        0        0        0        0        0       1

In [64]:
def ratio_page_all(universo, df_pageviews, mes_snapshot):
    campo = 'PAGE'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [106]:
def ratio_ccategory_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [107]:
def ratio_cctop_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_TOP'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [108]:
def ratio_ccbottom_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_BOTTOM'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_site_id_all(universo, df_pageviews, mes_snapshot):
    campo = 'SITE_ID'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_mobile_device_all(universo, df_devicedata, mes_snapshot):
    campo = 'IS_MOBILE_DEVICE'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)

In [ ]:
def ratio_connection_speed_all(universo, df_devicedata, mes_snapshot):
    campo = 'CONNECTION_SPEED'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)

In [ ]:
def ratio_on_site_search_term_all(universo, df_devicedata, mes_snapshot):
    campo = 'ON_SITE_SEARCH_TERM'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)